In [1]:
import numpy as np

In [ ]:
# Giriş değerlerinde negatif sayıları sıfırlayarak doğrusal olmayan bir aktivasyon fonksiyonu oluşturuyoruz
def relu(x):
    return np.maximum(0, x)

# ReLU fonksiyonunun türevini hesaplıyoruz. Giriş 0'dan büyükse 1, değilse 0 döndürüyoruz
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

class NeuralMultiplier:
    def __init__(self):
        # İlk gizli katmanımızda 16, ikinci gizli katmanımızda 8 nöron kullanıyoruz
        self.hidden1_size = 16
        self.hidden2_size = 8
        
        # Giriş katmanından ilk gizli katmana giden ağırlıkları Xavier/Glorot yöntemiyle başlatıyoruz
        # 2 giriş, hidden1_size kadar çıkış olacak şekilde ağırlık matrisimizi oluşturuyoruz
        self.W1 = np.random.randn(2, self.hidden1_size) * np.sqrt(2.0/2)
        # İlk gizli katman için bias değerlerimizi sıfır olarak başlatıyoruz
        self.b1 = np.zeros((1, self.hidden1_size))
        
        # İlk gizli katmandan ikinci gizli katmana giden ağırlıkları oluşturuyoruz
        self.W2 = np.random.randn(self.hidden1_size, self.hidden2_size) * np.sqrt(2.0/self.hidden1_size)
        # İkinci gizli katman için bias değerlerimizi sıfır olarak başlatıyoruz
        self.b2 = np.zeros((1, self.hidden2_size))
        
        # İkinci gizli katmandan çıkış katmanına giden ağırlıkları oluşturuyoruz
        self.W3 = np.random.randn(self.hidden2_size, 1) * np.sqrt(2.0/self.hidden2_size)
        # Çıkış katmanı için bias değerimizi sıfır olarak başlatıyoruz
        self.b3 = np.zeros((1, 1))
        
        # Ağırlık güncellemelerinde kullanacağımız öğrenme oranını belirliyoruz
        self.learning_rate = 0.001

    def forward(self, x1, x2):
        # Giriş değerlerimizi 0-1 aralığına normalize ediyoruz
        x1_norm = x1 / 10.0
        x2_norm = x2 / 10.0
        # Normalize edilmiş girişleri bir matrise dönüştürüyoruz
        self.input_layer = np.array([[x1_norm, x2_norm]])
        
        # İlk gizli katman çıktısını hesaplıyoruz: (giriş x ağırlıklar + bias)'a ReLU uyguluyoruz
        self.hidden1 = relu(np.dot(self.input_layer, self.W1) + self.b1)
        # İkinci gizli katman çıktısını hesaplıyoruz
        self.hidden2 = relu(np.dot(self.hidden1, self.W2) + self.b2)
        # Son katmanın çıktısını hesaplıyoruz (burada ReLU kullanmıyoruz)
        self.output = np.dot(self.hidden2, self.W3) + self.b3
        
        # Normalize edilmiş çıktıyı gerçek değer aralığına geri dönüştürüyoruz
        return self.output[0, 0] * 100

    def backward(self, target, predicted):
        # Giriş verisinin batch boyutunu alıyoruz
        batch_size = self.input_layer.shape[0]
        
        # Tahmin ile hedef arasındaki hatayı normalize ediyoruz
        error = (predicted - target) / 100

        # Gradyan, hatanın en hızlı arttığı yönü gösterir
        # Biz ağırlıkları gradyanın tersi yönünde güncelleriz (çünkü hatayı azaltmak istiyoruz)
        # Öğrenme oranı bu güncellemenin ne kadar büyük olacağını belirler
        
        # Çıkış katmanı için gradyanları hesaplıyoruz
        d_output = error
        # Çıkış katmanı ağırlık gradyanlarını hesaplıyoruz
        d_W3 = np.dot(self.hidden2.T, d_output)
        # Çıkış katmanı bias gradyanlarını hesaplıyoruz
        d_b3 = np.sum(d_output, axis=0, keepdims=True)
        
        # İkinci gizli katman için gradyanları hesaplıyoruz
        d_hidden2 = np.dot(d_output, self.W3.T) * relu_derivative(self.hidden2)
        # İkinci gizli katman ağırlık gradyanlarını hesaplıyoruz
        d_W2 = np.dot(self.hidden1.T, d_hidden2)
        # İkinci gizli katman bias gradyanlarını hesaplıyoruz
        d_b2 = np.sum(d_hidden2, axis=0, keepdims=True)
        
        # İlk gizli katman için gradyanları hesaplıyoruz
        d_hidden1 = np.dot(d_hidden2, self.W2.T) * relu_derivative(self.hidden1)
        # İlk gizli katman ağırlık gradyanlarını hesaplıyoruz
        d_W1 = np.dot(self.input_layer.T, d_hidden1)
        # İlk gizli katman bias gradyanlarını hesaplıyoruz
        d_b1 = np.sum(d_hidden1, axis=0, keepdims=True)
        
        # Hesapladığımız gradyanlar ile ağırlık ve bias değerlerini güncelliyoruz
        self.W3 -= self.learning_rate * d_W3
        self.b3 -= self.learning_rate * d_b3
        self.W2 -= self.learning_rate * d_W2
        self.b2 -= self.learning_rate * d_b2
        self.W1 -= self.learning_rate * d_W1
        self.b1 -= self.learning_rate * d_b1

    def train(self, epochs=2000):
        # Test verilerimizi önceden belirliyoruz ki rastgele sayılar bunlardan oluşmasın
        test_pairs = set([(6,7), (3,4), (5,5), (8,2), (9,9)])
        
        # 500 adet eğitim verisi oluşturuyoruz ama test verilerini dahil etmiyoruz
        train_size = 500
        train_pairs = []
        
        while len(train_pairs) < train_size:
            # 1-9 arası rastgele iki sayı seçiyoruz
            x1 = np.random.randint(1, 10)
            x2 = np.random.randint(1, 10)
            
            # Eğer seçilen sayı çifti test verilerinde yoksa, eğitim verilerine ekleyen kod
            if (x1, x2) not in test_pairs:
                train_pairs.append([x1, x2])
        
        # Numpy dizisine çeviriyoruz
        X = np.array(train_pairs)
        y = X[:, 0] * X[:, 1]  # çarpım sonuçlarını hesaplıyoruz
        
        best_error = float('inf')
        patience = 50
        patience_counter = 0
        
        # Belirlenen epoch sayısı kadar eğitim yapıyoruz
        for epoch in range(epochs):
            total_error = 0
            # Her seferinde 32'lik gruplar halinde eğitim yapıyoruz (mini-batch)
            batch_size = 32
            for i in range(0, len(X), batch_size):
                batch_X = X[i:i+batch_size]
                batch_y = y[i:i+batch_size]
                
                # Her batch içindeki örnekler için ileri ve geri yayılım yapıyoruz
                for j in range(len(batch_X)):
                    prediction = self.forward(batch_X[j][0], batch_X[j][1])
                    total_error += (batch_y[j] - prediction) ** 2
                    self.backward(batch_y[j], prediction)
            
            # Ortalama hatayı hesaplıyoruz
            avg_error = total_error / len(X)
            
            # Early stopping kontrolü yapıyoruz
            if avg_error < best_error:
                best_error = avg_error
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Belirli bir süre iyileşme olmazsa eğitimi durduruyor
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
            
            # Her 100 epoch'ta bir durumu raporluyoruz
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Ortalama Hata: {avg_error:.2f}")
                test_cases = [(2,3), (4,5), (3,7)]
                for x1, x2 in test_cases:
                    pred = self.forward(x1, x2)
                    print(f"{x1} x {x2} = {pred:.1f} (Gerçek: {x1*x2})")

    # Eğitilmiş modeli kullanarak çarpma işlemi yapıyoruz
    def multiply(self, x1, x2):
        return self.forward(x1, x2)

In [ ]:
# Modelden bir örnek (instance) oluşturuyoruz ve eğitiyoruz
print("Eğitim başlıyor...")
multiplier = NeuralMultiplier()
multiplier.train()

In [ ]:
# Test verilerimiz ile modelimizin performansını ölçüyoruz.
print("\nTest sonuçları:")
test_cases = [(6,7), (3,4), (5,5), (8,2), (9,9)]
total_error = 0

for a, b in test_cases:
    sonuc = multiplier.multiply(a, b)
    gercek = a * b
    hata = abs(gercek - sonuc)  # Mutlak hata
    yuzde_hata = (hata / gercek) * 100  # Yüzde hata
    
    print(f"{a} x {b} = {sonuc:.1f} (Gerçek: {gercek}, Hata: {hata:.2f}, Hata Yüzdesi: %{yuzde_hata:.2f})")
    total_error += hata

# Ortalama hata hesapla
avg_error = total_error / len(test_cases)
print(f"\nOrtalama Mutlak Hata: {avg_error:.2f}")